In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST_data/", one_hot = True)
logs_path = 'tensorboard/mnist_cnn/'

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)
num_channels = 1 #grayscale
num_classes = 10

In [4]:
def new_weights(shape): return tf.Variable(tf.truncated_normal(shape, stddev=0.03))

def new_biases(length): return tf.Variable(tf.constant(0.03, shape=[length]))

In [5]:
def new_conv_layer(input, num_input_channels, filter_size, filter_out): 

    #weight  input        input        input channels      output channel
    shape = [filter_size, filter_size, num_input_channels, filter_out]

    weights = new_weights(shape=shape)

    biases = new_biases(length=filter_out)

    layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

    layer += biases

    layer = tf.nn.relu(layer)
    print('input {}'.format(input))
    print('layer_out {}\nWeights {}'.format(layer, weights))
    return layer, weights
    
    #tf.nn.conv2d([-1, 28, 28, 1], [5, 5, 1, 32], strides=[1,1,1,1], use_cudnn_on_gpu=True,padding='SAME')    

In [6]:
def maxpool2d(input, size, move):
    
    layer = tf.nn.max_pool(input, ksize=[1,size,size,1], strides=[1,move,move,1], padding='VALID')
    
    print('input {}'.format(input))
    print('maxpool_out {}'.format(layer))
    return layer
    #performing relu after maxpooling reduces compute time. the result is the same if preformed in conv layer

In [7]:
def flatten_layer(layer):
   
    layer_shape = layer.get_shape()

    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])

    print('flat_layer {}'.format(num_features))
    return layer_flat, num_features

In [8]:
def new_fc_layer(input, num_inputs, num_outputs, use_relu=True): 

    weights = new_weights(shape=[num_inputs, num_outputs])
    
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases

    print('input {}'.format(input))
    print('fc_out {}\nWeight {}'.format(layer, weights))

    if use_relu: layer = tf.nn.relu(layer)

    return layer

In [9]:
with tf.name_scope('Image_Placeholder'):
    x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

with tf.name_scope('Label_Placeholder'):
    y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

with tf.name_scope('Image'):
    x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

with tf.name_scope('True_Class'):
    y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [10]:
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, 
                                            num_input_channels=1,
                                            filter_size=5,
                                            filter_out=32)

maxp_layer1 = maxpool2d(layer_conv1, size=3, move=1)

with tf.name_scope('convet_1'):
    tf.summary.histogram('layer_1', layer_conv1)
    tf.summary.histogram('Weight_1', weights_conv1)
    
with tf.name_scope('Maxpool_1'):
    tf.summary.histogram('Maxpool_1', maxp_layer1)

input Tensor("Image/Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
layer_out Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Weights <tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>
input Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
maxpool_out Tensor("MaxPool:0", shape=(?, 26, 26, 32), dtype=float32)


In [11]:
layer_conv2, weights_conv2 = new_conv_layer(input=maxp_layer1,
                                            num_input_channels=32,
                                            filter_size=3,
                                            filter_out=64)

maxp_layer2 = maxpool2d(input=layer_conv2, size=2, move=2)

with tf.name_scope('convet_2'):
    tf.summary.histogram('layer_2', layer_conv2)
    tf.summary.histogram('Weight_2', weights_conv2)
    
with tf.name_scope('Maxpool_2'):
    tf.summary.histogram('Maxpool_2', maxp_layer2)

input Tensor("MaxPool:0", shape=(?, 26, 26, 32), dtype=float32)
layer_out Tensor("Relu_1:0", shape=(?, 26, 26, 64), dtype=float32)
Weights <tf.Variable 'Variable_2:0' shape=(3, 3, 32, 64) dtype=float32_ref>
input Tensor("Relu_1:0", shape=(?, 26, 26, 64), dtype=float32)
maxpool_out Tensor("MaxPool_1:0", shape=(?, 13, 13, 64), dtype=float32)


In [12]:
layer_conv3, weights_conv3 = new_conv_layer(input=maxp_layer2,
                                            num_input_channels=64,
                                            filter_size=3,
                                            filter_out=128)

maxp_layer3 = maxpool2d(input=layer_conv3, size=2, move=2)

with tf.name_scope('convet_3'):
    tf.summary.histogram('layer_3', layer_conv3)
    tf.summary.histogram('Weight_3', weights_conv3)
    
with tf.name_scope('Maxpool_3'):
    tf.summary.histogram('Maxpool_3', maxp_layer3)

input Tensor("MaxPool_1:0", shape=(?, 13, 13, 64), dtype=float32)
layer_out Tensor("Relu_2:0", shape=(?, 13, 13, 128), dtype=float32)
Weights <tf.Variable 'Variable_4:0' shape=(3, 3, 64, 128) dtype=float32_ref>
input Tensor("Relu_2:0", shape=(?, 13, 13, 128), dtype=float32)
maxpool_out Tensor("MaxPool_2:0", shape=(?, 6, 6, 128), dtype=float32)


In [13]:
layer_flat, num_features = flatten_layer(maxp_layer2)

flat_layer 10816


In [14]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=512)

with tf.name_scope('Fully_Connctd_1'):
    tf.summary.histogram('fc_1', layer_fc1)

input Tensor("Reshape:0", shape=(?, 10816), dtype=float32)
fc_out Tensor("add_3:0", shape=(?, 512), dtype=float32)
Weight <tf.Variable 'Variable_6:0' shape=(10816, 512) dtype=float32_ref>


In [15]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=512,
                         num_outputs=num_classes,
                         use_relu=False)
with tf.name_scope('Fully_Connctd_2'):
    tf.summary.histogram('fc_2', layer_fc2)

input Tensor("Relu_3:0", shape=(?, 512), dtype=float32)
fc_out Tensor("add_4:0", shape=(?, 10), dtype=float32)
Weight <tf.Variable 'Variable_8:0' shape=(512, 10) dtype=float32_ref>


In [16]:
with tf.name_scope('Y_pred_Softmaxe'):
    y_pred = tf.nn.softmax(layer_fc2)

with tf.name_scope('Y_pred_Class_argmax'):
    y_pred_cls = tf.argmax(y_pred, dimension=1)

with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)

with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('Acc', accuracy)

In [17]:
with tf.name_scope('Cross_Entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)
    #tf.summary.scalar('entropy', cross_entropy)

with tf.name_scope('Cost'):
    cost = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('Loss', cost)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
merged_summary_op = tf.summary.merge_all()

In [19]:
def optimize(num_iterations):

    global total_iterations

    start_time = time.time()

    for i in range(total_iterations,total_iterations + num_iterations ):

        x_batch, y_true_batch = data.train.next_batch(train_batch_size)

        feed_dict_train = {x: x_batch, y_true: y_true_batch}

        _ = session.run([optimizer], feed_dict=feed_dict_train)
        #summary_writer.add_summary(summary, i)

        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc, summary = session.run([accuracy, merged_summary_op], feed_dict=feed_dict_train)
            print('iteration: {}   Training Accuracy: {}'.format(i, acc))
            summary_writer.add_summary(summary, i)


    print('End ==== iteration: {0:>6}   Training Accuracy: {1:>6.1%}'.format(i, acc))
    total_iterations += num_iterations

    end_time = time.time()

    time_dif = end_time - start_time

    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [20]:
if(tf.Session()._closed==False): 
    session = tf.Session()
    session.run(tf.global_variables_initializer())
else : print('session already running')
    
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [21]:
total_iterations = 0
train_batch_size = 256
optimize(num_iterations=1000)

iteration: 0   Training Accuracy: 0.1328125
iteration: 100   Training Accuracy: 0.95703125
iteration: 200   Training Accuracy: 0.984375
iteration: 300   Training Accuracy: 0.98046875
iteration: 400   Training Accuracy: 0.98828125
iteration: 500   Training Accuracy: 0.98046875
iteration: 600   Training Accuracy: 0.99609375
iteration: 700   Training Accuracy: 0.9921875
iteration: 800   Training Accuracy: 0.9921875
iteration: 900   Training Accuracy: 0.99609375
End ==== iteration:    999   Training Accuracy:  99.6%
Time usage: 0:00:36


In [22]:
test_batch_size = 1000
n_batches = data.test.images.shape[0] // test_batch_size
pred = []
def print_test_accuracy():
    cumulative_accuracy = 0.0
    start_time = time.time()
    for index in range(n_batches):

        print('{} -- of run size {}'.format(index+1, n_batches))
        batch = data.test.next_batch(test_batch_size)

        images = batch[0]
        labels = batch[1]
        feed_dict = {x: images, y_true: labels}

        cumulative_accuracy += session.run(accuracy, feed_dict)
        temp = session.run(y_pred_cls, feed_dict)
        pred.append(temp)
        print("batch accuracy {}".format(cumulative_accuracy / (index+1) ) )
    end_time = time.time()
    time_dif = end_time - start_time
    print("test accuracy {} in Time {}".format(cumulative_accuracy / n_batches, (timedelta(seconds=int(round(time_dif)))) ) )

In [23]:
print_test_accuracy()

1 -- of run size 10
batch accuracy 0.9940000176429749
2 -- of run size 10
batch accuracy 0.9945000112056732
3 -- of run size 10
batch accuracy 0.9943333466847738
4 -- of run size 10
batch accuracy 0.9930000156164169
5 -- of run size 10
batch accuracy 0.9928000092506408
6 -- of run size 10
batch accuracy 0.9930000106493632
7 -- of run size 10
batch accuracy 0.992428583758218
8 -- of run size 10
batch accuracy 0.9926250129938126
9 -- of run size 10
batch accuracy 0.992333345943027
10 -- of run size 10
batch accuracy 0.9913000106811524
test accuracy 0.9913000106811524 in Time 0:00:01


In [24]:
import os
from tensorflow.contrib.tensorboard.plugins import projector
import pandas as pd


metadata = os.path.join(logs_path, 'metadata.tsv')
plables = np.reshape(pred, (-1))
plables = np.asarray(pd.get_dummies(plables))

with open(metadata, 'w') as metadata_file:
    for row in range(10000):
        c = np.nonzero(plables[::1])[1:][0][row]
        metadata_file.write('{}\n'.format(c))
        
images = tf.Variable(data.test.images, name='images')
saver = tf.train.Saver([images])
session.run(images.initializer)
saver.save(session, os.path.join(logs_path, 'images.ckpt'))
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = images.name
embedding.metadata_path = metadata

projector.visualize_embeddings(tf.summary.FileWriter(logs_path), config)

In [ ]:
#tensorboard --logdir=tensorboard/mnist_cnn --port=8080
#embeddings do not work on the default port in this case

In [25]:
print(1)

1
